<a href="https://colab.research.google.com/github/dlwogns/ML-DL/blob/main/GoDaddy%20-%20Microbusiness%20Density%20Forecasting/1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### train

cflips - 연방정보처리 과정에 따른 각각의 카운티의 고유 넘버. 앞 두자리는 주(state), 뒤 세자리는 카운티. <br>
first day of month - 달의 첫날.<br>
microbusiness density - 각 카운티의 100명당 microbusiness 지수. 이 지표에 사용된 인구수는 2년씩 지연되어있음. (21년 density는 19년의 인구를 사용.) <br>
active - 각 카운티의 microbusiness raw count를 나타냄.<br>

### sample_submission
target -> microbusiness density<br>

### test

row_id, cfips, first day of month 만 제공. -> 시계열

### census starter
관련한 정보들 모음.<br>
이것도 또한 2년의 지연이 있음.<br>
pct_bb -> 카운티 인터넷가입 가정 비율 <br>
pct_college -> 인구수 대비 25세 이상 4년제 대학 학생 비율.<br>

pct foreign born -> 미국 외에서 태어난 사람 비율. 인구대비<br>
pct it workers -> 노동자 중 it산업 종사자 비율<br>
median hh inc -> 카운티 가장 수입 중앙값

# aim to
19년 1월 ~ 22년 10월까지의 정보가 주어지고, 22년 11월 ~ 23년 1월까지 예측

# reference
https://www.kaggle.com/code/kimtaehun/complete-baseline-code-with-various-ml-model<br>
baseline https://www.kaggle.com/code/masahiroichigo/godaddy-microbusiness-density-forecasting-with-tf<br>

In [12]:
train = pd.read_csv("/content/drive/MyDrive/data/GoDaddy/train.csv")
test = pd.read_csv("/content/drive/MyDrive/data/GoDaddy/test.csv")
census = pd.read_csv("/content/drive/MyDrive/data/GoDaddy/census_starter.csv")
sample_submission = pd.read_csv("/content/drive/MyDrive/data/GoDaddy/sample_submission.csv")

In [10]:
train.nunique()

row_id                   122265
cfips                      3135
county                     1871
state                        51
first_day_of_month           39
microbusiness_density     97122
active                    19193
dtype: int64

In [11]:
test.nunique()

row_id                25080
cfips                  3135
first_day_of_month        8
dtype: int64

39개의 데이터를 받아서, 8개를 나타내야됨.

census에는 있지만, train에는 없는 것을 제외

In [14]:
cen = census['cfips'].unique()
tra = train['cfips'].unique()
extra = []
for c in cen:
  if c not in tra:
    extra.append(c)
extra

[2158, 2261, 31117, 46102, 51580, 51640, 51690]

In [15]:
extra_index = []
for c in extra:
  idx = census[census['cfips'] == c].index[0]
  extra_index.append(idx)
census.iloc[extra_index]

,pct_bb_2017,pct_bb_2018,pct_bb_2019,pct_bb_2020,pct_bb_2021,cfips,pct_college_2017,pct_college_2018,pct_college_2019,pct_college_2020,...,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021
81,58.4,62.5,65.3,66.1,70.0,2158,3.0,3.3,2.8,2.6,...,1.7,1.9,1.2,1.1,0.9,36468,35539.0,36754,37358.0,37975.0
92,80.5,79.1,80.4,NaN,NaN,2261,23.1,19.0,16.5,NaN,...,3.3,3.9,5.3,NaN,NaN,86019,82306.0,79867,NaN,NaN
1712,62.0,59.6,62.8,68.8,78.6,31117,19.6,18.0,17.6,18.5,...,3.8,0.0,0.0,0.0,0.0,60714,48882.0,48125,51932.0,53750.0
2412,44.8,48.5,45.2,53.0,56.5,46102,9.5,8.7,7.3,6.8,...,0.5,0.6,0.0,0.0,0.0,27804,30347.0,31997,31423.0,33192.0
2921,59.8,69.0,77.2,80.1,82.9,51580,8.7,10.3,10.4,10.3,...,2.0,1.9,1.6,0.4,0.4,35546,39432.0,40655,41024.0,41242.0
2928,63.1,66.8,68.5,75.4,81.7,51640,5.7,7.4,8.3,9.7,...,1.1,0.6,1.5,0.8,0.8,31311,31938.0,33575,35184.0,39808.0
2936,63.4,72.1,73.9,76.6,80.2,51690,13.1,14.1,14.2,14.5,...,1.4,1.7,1.7,1.6,0.3,33545,33224.0,34371,36166.0,36832.0


In [16]:
census.drop(extra_index, axis=0, inplace = True)
len(census)

3135

In [20]:
pd.merge(train, census, how='inner', on='cfips')
# on은 기준열을 잡아주는 것으로, default는 None이고 이 경우에는 index를 기준으로 한다.

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,pct_bb_2017,pct_bb_2018,pct_bb_2019,...,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,76.6,78.9,80.6,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,76.6,78.9,80.6,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,76.6,78.9,80.6,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,76.6,78.9,80.6,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,76.6,78.9,80.6,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,71.1,73.3,76.8,...,0.6,0.6,0.0,0.0,0.0,59605,52867.0,57031,53333.0,65566.0
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,71.1,73.3,76.8,...,0.6,0.6,0.0,0.0,0.0,59605,52867.0,57031,53333.0,65566.0
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,71.1,73.3,76.8,...,0.6,0.6,0.0,0.0,0.0,59605,52867.0,57031,53333.0,65566.0
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,71.1,73.3,76.8,...,0.6,0.6,0.0,0.0,0.0,59605,52867.0,57031,53333.0,65566.0


In [22]:
train["first_day_of_month"] = pd.to_datetime(train["first_day_of_month"])
train = train.sort_values(["cfips", "first_day_of_month"]).reset_index(drop = True)

test["first_day_of_month"] = pd.to_datetime(test["first_day_of_month"])
test = test.sort_values(["cfips", "first_day_of_month"]).reset_index(drop = True)

In [25]:
import tensorflow as tf

### Time Series Windowing Data
https://albertum.medium.com/preprocessing-time-series-to-windowed-datasets-a464799b1df7

In [23]:
HORIZON = 8
WINDOW = 8

def get_labelled_window(x, horizon = HORIZON):
  return x[:, :-horizon], x[:, -horizon]

def make_windows(x, window_size=WINDOW, horizon = HORIZON):
  window_step = np.expand_dims(np.arange(window_size+horizon), axis=0)
  # arange -> 0 ~ argument까지의 일정한 step의 1차원 배열 return.
  # expand_dims -> [1,2], axis=0 -> [[1,2]] [1,2], axis=1 -> [[1],[2]]
  # 차원을 확대하는 것
  # time series를 위한 windowing data를 만들기 위한 
  window_indexes = window_step + np.expand_dims(np.arange(len(x) - (window_size + horizon-1)), axis=0).T
  print(f'Window indexes: \n{window_indexes, window_indexes.shape}')

  windowed_array = x[window_indexes]

  windows, labels = get_labelled_window(windowed_array, horizon = horizon)

  return windows, labels

In [24]:
def make_train_test_split(windows, labels, split_size=1):
  split_size = len(windows) - split_size
  train_windows = windows[:split_size]
  train_labels = labels[:split_size]
  test_windows = windows[split_size:]
  test_labels = labels[split_size:]
  return train_windows, test_windows, train_labels, test_labels

In [26]:
def evaluate_preds(y_true, y_pred):
  y_true = tf.cast(y_true, dtype = tf.float32)
  y_pred = tf.cast(y_pred, dtype = tf.float32)
  mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)[0]
  mse = tf.keras.metrics.mean_squared_error(y_true, y_pred)[0]
  rmse = tf.sqrt(mse)
  mape = tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred)[0]

  return {"mae" : mae.numpy(),
          "mse" : mse.numpy(),
          "rmse" : rmse.numpy(),
          "mape" : mape.numpy()}

In [27]:
def make_preds(model, input_data):
  forecast = model.predict(input_data)
  return tf.squeeze(forecast)

In [28]:
IDS = train.cfips.unique()
c = np.random.choice(IDS)
c

13219

In [31]:
a = train[train['cfips'] == c]
last = a.microbusiness_density.values[-1]
windows, labels = make_windows(a.microbusiness_density.values)
len(windows), len(labels), last

Window indexes: 
(array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
       [ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17],
       [ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
       [ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
       [ 6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
       [ 7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22],
       [ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
       [ 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25],
       [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26],
       [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27],
       [13, 14, 15, 16, 17, 18, 1

(24, 24, 7.8661437)

In [32]:
a

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
19188,13219_2019-08-01,13219,Oconee County,Georgia,2019-08-01,8.207680,2161
19189,13219_2019-09-01,13219,Oconee County,Georgia,2019-09-01,8.359604,2201
19190,13219_2019-10-01,13219,Oconee County,Georgia,2019-10-01,8.336815,2195
19191,13219_2019-11-01,13219,Oconee County,Georgia,2019-11-01,8.431767,2220
19192,13219_2019-12-01,13219,Oconee County,Georgia,2019-12-01,8.454555,2226
19193,13219_2020-01-01,13219,Oconee County,Georgia,2020-01-01,8.248717,2234
19194,13219_2020-02-01,13219,Oconee County,Georgia,2020-02-01,8.293025,2246
19195,13219_2020-03-01,13219,Oconee County,Georgia,2020-03-01,8.315179,2252
19196,13219_2020-04-01,13219,Oconee County,Georgia,2020-04-01,8.296718,2247
19197,13219_2020-05-01,13219,Oconee County,Georgia,2020-05-01,8.437027,2285


In [33]:
train_windows, test_windows, train_labels, test_labels = make_train_test_split(windows, labels)

In [34]:
test_labels

array([7.886972])

In [35]:
from tensorflow.keras import layers

In [ ]:
model = tf.keras.Sequential()